---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

# Load Libraries

In [30]:
# Libraries
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Data Preprocessing
## Data Quality Assessment
A data quality assessment was conducted to determine the degree of data cleaning, transformation, and data reduction that would be required. A cursory and unstructured analysis revealed:
1. A non-trivial degree of spelling errors were found in the address fields. Any analysis based upon city would require significant preprocessing to correct misspellings, abbreviations which were found in approximately 10% of the observations. 
2. Hearing date was missing for both compliant and non-compliant observations and in some cases preceded the ticket_issue
date. 
4. Compliance values were missing for judgments in which the alleged violator was found to be not responsible. This would require some data reduction to illiminate those observations. 

## Data Cleaning
The intent at this stage was to address some of the data quality issues, rule-out variables not useful to the analysis, and to remove missing data that could not be impuned.

### Data Reduction
The following observations were removed from the data set.
1. The observations in which the alleged violator was judged to be 'not responsible by disposition' were removed from the data set. This reduced the data set from 250,306 to 159,880 observations. 
2. Observations with missing hearing dates also removed further reducing the available training data to an approximate 159,700 observations.
3. Lastly, observations in which the hearing dates were before the ticket issue date were excised from the data, leaving a total training set size of 159,572 observations.

### Feature Selection
The following variables were removed from the data set for reasons of redundancy, lack of association with the target variable, or other practical concerns.
- ticket_id - used for indexing purposes only
- violator_name - Too many levels and could lead to overfitting.
- violation_street_number - No association with compliance
- violation_zip_code - all blanks
- mailing_address_str_number - No association with compliance
- mailing_address_str_name - Frought with misspellings.  Zip code was a better proxy for location
- non_us_str_code and country were removed given the extreme imbalance in the data set.
- violation_description - directly correlated with violation_code, which was a higher quality indicator
- fine_amount - included in judgement amount (unless alleged violator deemed not responsible)
- admin_fee - included in judgement amount
- state_fee - included in judgement amount
- late_fee - included in judgement amount
- discount_amount - included in judgment amount
- clean_up_cost - correlated with judgment amount
- grafitti_status - correlated with judgment amount

Therefore, the following variables from the original training set would be included for further analysis.
1. agency_name
2. inspector_name
3. state
4. zip_code
5. ticket_issued_date
6. hearing_date
7. violation_code
8. judgment_amount
9. compliance

Variables 10, 11, 12 and 13 would be used in the next step to derive new features.

### Data Integration and Feature Engineering
The following data integration tasks provided the following additional derived features.
- Address and latitude / longitude data would be merged into the overall database
- payment_window the number of days between ticket_issued_date and hearing_date
- amt_per_day the judgment_amount divided by the payment_window
- mean_compliance_zip_code 


In [43]:
def preprocess(what, df, addresses, latlong):    
    
    # Data Reduction (training set only)    
    if (what == 'train'):    
        # Data Reduction       
        df = df[df.compliance.notnull()]
        df = df[df.hearing_date.notnull()]
        df = df.loc[df['hearing_date'] > df['ticket_issued_date']]
        
    # Feature Selecction
    if (what == 'train'):
        features = ['ticket_id', 'agency_name', 'inspector_name', 'state', 'zip_code', 'ticket_issued_date',
                    'hearing_date', 'violation_code', 'judgment_amount', 'compliance']    
    else:
        features = ['ticket_id', 'agency_name', 'inspector_name', 'state', 'zip_code', 'ticket_issued_date',
                    'hearing_date', 'violation_code', 'judgment_amount']    
    df = df[features]
    
    # Feature Engineering
    df = pd.merge(df, addresses, how = 'left', on = ['ticket_id'])
    df = pd.merge(df, latlong, how = 'left', on = ['address'])    

    # Payment window and amount per day
    df['payment_window'] = (pd.to_datetime(df['hearing_date']) - pd.to_datetime(df['ticket_issued_date'])).dt.days
    df['amt_per_day'] = df['judgment_amount'] / df['payment_window']

    # Mean compliance by zip code
    if (what == 'train'):        
        df['compliance_by_zip'] = df.groupby(['zip_code'])['compliance'].mean()
    print(df.info())
    #print(df.info())
    #compliance_by_zip_code = pd.DataFrame(zip_code = df.groupby(['zip_code'])['compliance'].mean())
    #print(compliance_by_zip_code.info())
    #df = pd.merge(df, compliance_by_zip_code, how = 'left', on = ['zip_code'])
    
    # Replace NaNs with zeros
    df=df.fillna(0)
    df = df.set_index('ticket_id')    
    
    # Convert categorical variables to numeric
    le = LabelEncoder()
    categorical = list(df.select_dtypes(include=['object']).columns.values)
    for cat in categorical:
        df[cat] = le.fit_transform(df[cat].astype(str))

    return(df)

    
# Obtain data
train = pd.read_csv('train.csv', encoding = "Latin-1", low_memory = False)
test  = pd.read_csv('test.csv', encoding = "Latin-1", low_memory = False)
addresses = pd.read_csv('addresses.csv')
latlong   = pd.read_csv('latlons.csv')

# Preprocess data
train = preprocess('train', train, addresses, latlong)
test = preprocess('test', test, addresses, latlong)

# Create feature matrices and target vector
X_train = train.drop("compliance", axis = 1)
y_train = train['compliance']

X_train

zip_code
0             0.069307
10001         0.000000
10004         0.000000
10005         0.000000
10010         0.100000
10011         0.333333
10013         0.000000
10016         0.444444
10017         0.181818
10018         0.000000
10019         0.000000
1002          0.000000
10020         0.000000
10022         0.000000
10025         0.200000
10027         1.000000
10028         0.000000
10029         0.000000
10030         0.333333
10031         0.023810
10032         0.000000
10035         0.000000
10036         0.000000
100364003     0.000000
10040         0.000000
10043         0.000000
1005          0.000000
10106         0.000000
10118         0.000000
10120         0.000000
                ...   
98660         0.000000
98703         0.000000
98807         0.000000
99037         0.111111
99071         0.000000
99127         0.000000
99201         0.000000
99214         0.000000
99307         0.166667
9947          0.000000
99514         0.000000
99516         0.000000
99

KeyError: 'Column not found: compliance'